# Final Model Prediction

In [3]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
import pandas as pd
import json
import numpy as np
import random
from keras.models import load_model
import os
import tensorflow as tf

## loding the model

In [4]:
model = load_model('FinalModelPrediction.hdf5')

Loding Data

In [ ]:
df = pd.read_csv('D:/+DATA+/allok_w15_nosell.csv',index_col=0)
high_weight=3
df["high"]=(df["open"]+high_weight*df["high"]+df["low"]+df["close"])/(3+high_weight)
df.rename(columns={"high":"price"},inplace = True)
df["BTC_high"]=(df["BTC_open"]+high_weight*df["BTC_high"]+df["BTC_low"]+df["BTC_close"])/(3+high_weight)
df.rename(columns={"BTC_high":"BTC_price"},inplace = True)
df=df.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])


In [ ]:
df.iloc[0]

Persentage Nomalization

In [5]:
for key in df.keys():
    if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["BTC_price"]-df[key])/df["BTC_price"]
    if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
        df[key]=(df["price"]-df[key])/df["price"]

NameError: name 'df' is not defined

Import Normalization values

In [6]:
def normalize(dataset,file='Normalization_Values_Final.json'):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization==None
    if(Normalization==None):
        print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        print('normalization is loaded')

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)


In [ ]:
dt=df.drop(columns=["buy"]).to_numpy()

In [ ]:
Y=df.drop(columns=["buy".to_numpy()])

In [7]:
def Buy_Dessision(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Normalized(input):
    A=np.array(input)
    A = A.reshape(1,A.shape[0])
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded[0])

def Buy_Dessision_Multi_In_Out(input):
    A=np.array(input)
    predictions = model.predict(normalize(A))
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

def Buy_Dessision_Multi_In_Out_Normalized(input):
    A=np.array(input)
    predictions = model.predict(A)
    rounded = [round(x[0]) for x in predictions]
    return(rounded)

In [ ]:
Buy_Dessision(df.drop(columns=["buy"]).iloc[111520])

In [142]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [147]:
with tf.device('/cpu:0'):
    accuracy = model.evaluate(dt, Y)
print("accuracy: "str(accuracy[1]))


33887/33887 [==============================] - 411s 12ms/step - loss: 0.3554 - accuracy: 0.9220


In [22]:
import ccxt
Exchange=ccxt.binance()
ex=Exchange

In [27]:
bitcoin_ticker = Exchange.fetch_ticker('BTC/USDT')
bitcoin_ticker['info']['askPrice']


'18385.83000000'

'18398.77000000'

In [38]:
ohlcv = ex.fetch_ohlcv('BTC/USDT', '1m', limit=15)
ohlcv = ex.fetch_ohlcv('BTC/USDT', '5m', limit=15)
ohlcv = ex.fetch_ohlcv('BTC/USDT', '15m', limit=15)
ohlcv = ex.fetch_ohlcv('BTC/USDT', '1h', limit=15)
ohlcv = ex.fetch_ohlcv('BTC/USDT', '1d', limit=15)

np.array(ohlcv)[:,5]

array([1470.89817,  956.71927,  964.05919,  799.82492,  966.49763,
        715.62772, 1174.58647,  931.53968, 1227.95439, 1333.94053,
       1119.03133, 1114.87588, 1809.99656, 1672.69011,  321.70569])